In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import numpy as np
import cv2

In [2]:
def to_canny(img, kernel_size, low_threshold, high_threshold):
    """
    'img' should be original image to be transformed
    
    Returns and image with Canny edges
    """
    gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    # Define a kernel size for Gaussian smoothing / blurring
    blur_gray = cv2.GaussianBlur(gray,(kernel_size, kernel_size),0)
    edges = cv2.Canny(blur_gray, low_threshold, high_threshold)
    return edges

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane. 
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    slope = 0
    left_slope = []
    right_slope = []
    left_lines = []
    right_lines = []
    #Get the Left/Right Lines and Slopes
    for line in lines:
        for x1,y1,x2,y2 in line:
            slope = ((y2-y1)/(x2-x1))
            if slope > 0:
                right_slope.append(slope)
                right_lines.append((x1,y1))
            else:
                left_slope.append(slope)
                left_lines.append((x2,y2))
    
    #Get the average slopes
    ave_right_slope = sum(right_slope)/len(right_slope)
    ave_left_slope = sum(left_slope)/len(left_slope)
    
    #Get left line
    left_y1 = min([line[1] for line in left_lines])
    left_x1=0
    #Loop through to get the last x point corresponding to minimum y, in case there the multiple x for same min y
    for line in left_lines:
        if line[1] == left_y1:
            left_x1 = line[0]
    left_pair = tuple([left_x1] + [left_y1])
    
    #Get the Right line
    right_y1 = min([line[1] for line in right_lines])
    right_x1=0
    #Loop through to get the last x point corresponding to minimum y, in case there the multiple x for same min y
    for line in right_lines:
        if line[1] == right_y1:
            right_x1 = line[0]
    right_pair = tuple([right_x1] + [right_y1])

    left_x = int((img.shape[1]-left_pair[1])/ave_left_slope) + left_pair[0]
    right_x = int((img.shape[1]-right_pair[1])/ave_right_slope) + right_pair[0]
    
    cv2.line(img, left_pair, (left_x, img.shape[1]), color, thickness)
    cv2.line(img, right_pair, (right_x, img.shape[1]), color, thickness)  
    
        
def to_hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    'img' should be transformed in Canny edges
    
    Returns an image after Hough transformation
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines, thickness=10)
    return line_img


def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

# Display the image
def process_image(image):
    o_image = np.copy(image)
    line_edges = to_canny(image, 5, 50, 150)

    imshape = image.shape
    l_vertices = np.array([[(0,imshape[0]),(480,317),(480,317),(imshape[1],imshape[0])]],dtype=np.int32)
    m_image = region_of_interest(line_edges, l_vertices)

    hl_image = to_hough_lines(m_image, 2, np.pi/180, 15, 30, 20)

    final_image = weighted_img(img=hl_image,initial_img=o_image)

    return final_image
    
    

image = mpimg.imread('test_images/solidWhiteRight.jpg')
plt.imshow(process_image(image))
plt.show()

[MoviePy] >>>> Building video extra.mp4
[MoviePy] Writing video extra.mp4


100%|████████████████████████████████████████████████████████████████████████████████| 251/251 [00:05<00:00, 46.91it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: extra.mp4 

Wall time: 6.31 s
